In [ ]:
import fastf1 as ff1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Rectangle
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [ ]:
plt.style.use('dark_background')
sns.set_palette("husl")
current_year = 2025
schedule = ff1.get_event_schedule(current_year)
print("Available 2025 F Events:")
print(schedule[['RoundNumber','EventName','EventDate','EventFormat']].head(19))


In [ ]:
# SELECT AND LOAD DATA
YEAR = 2025
ROUND = 1
try:
    event = ff1.get_event(YEAR,ROUND)
    print(f"Loading data for: {event['EventName']} ({event['EventDate']})")
    race = event.get_race()
    race.load()
    print(f"Race loaded succecssfully")
    print(f"Number of laps in race: {race.total_laps}")
    print(f"Number of drivers: {len(race.drivers)}")
except Exception as e:
    print(f"Error loading race data: {e}")
    print("Trying with different round...")

In [ ]:
# COMPLEX DATA PROCESSING

def get_position_data(race_session):
    drivers = race_session.drivers
    driver_info = {}
    for driver in drivers:
        driver_data = race_session.get_driver(driver)
        driver_info[driver] = {
            'full_name': f"{driver_data['FirstName']} {driver_data['LastName']}",
            'abbreviation': driver_data['Abbreviation'],
            'team': driver_data['TeamName'],
            'color': driver_data['TeamColor']
        }
    position_data = []
    max_laps = race_session.total_laps
    for lap_num in range(1,max_laps+1):
        lap_positions = {}
        for driver in drivers:
            try:
                driver_laps = race_session.laps.pick_driver(driver)
                if not driver_laps.empty and lap_num <= len(driver_laps):
                    lap_data = driver_laps.iloc[lap_num-1]
                    position = lap_data['Position']
                    if pd.notna(position):
                        lap_positions[driver] = int(position)
                    else:
                        if lap_num>1 and position_data:
                            last_pos = None
                            for prev_lap in reversed(position_data):
                                if driver in prev_lap['positions']:
                                    last_pos = prev_lap['positions'][driver]
                                    break
                            lap_positions[driver] = last_pos if last_pos else 20
                        else:
                            lap_positions[driver] = 20
            except (IndexError,KeyError):
                lap_positions[driver] = 20
        position_data.append({
            'lap': lap_num,
            'positions': lap_positions
        })
    
    return position_data, driver_info

try:
    position_data,driver_info = get_position_data(race)
    print("Position Data processed successfully")
    print(f"Data available for {len(position_data)} laps")
except Exception as e:
    print(f"Error processing data {e}")
                

In [ ]:
# ANIMATION FUNCTION

def create_race_animation(position_data,driver_info,event_name):
    fig,ax = plt.subplots(figsize=(14,10))
    fig.patch.set_facecolor('black')
    all_drivers = list(driver_info.keys())
    colors = {}
    color_palette = plt.cm.Set3(np.linspace(0,1,len(all_drivers)))

    for i, driver in enumerate(all_drivers):
        team_color = driver_info[driver].get('color',None)
        if team_color and team_color.startswith('#'):
            colors[driver] = team_color
        else:
            colors[driver] = color_palette[i]
    
    lines = {}
    for driver in all_drivers:
        line, = ax.plot([],[],linewidth=3,label=driver_info[driver]['abbreviation'],color=colors[driver],alpha=0.8)
        lines[driver] = line
    
    ax.set_xlim(0,len(position_data))
    ax.set_ylim(0.5,20.5)
    ax.set_xlabel('Lap Number',fontsize=12,color='white')
    ax.set_ylabel('Position',fontsize=12,color='white')
    ax.set_title(f'{event_name} - Race Positions by Lap',fontsize=16,color='white',pad=20)
    ax.invert_yaxis()

    ax.grid(True,alpha=0.3)
    ax.tick_params(colors='white')
    ax.spines['bottom'].set_color('white')
    ax.spines['top'].set_color('white')
    ax.spines['right'].set_color('white')
    ax.spines['left'].set_color('white')
    ax.legend(bbox_to_anchor=(1.05,1),loc='upper left',frameon=True,facecolor='black',edgecolor='white')

    def animate(frame):
        for driver in all_drivers:
            x_data = []
            y_data = []
            for lap_idx in range(min(frame+1,len(position_data))):
                lap_data = position_data[lap_idx]
                if driver in lap_data['positions']:
                    x_data.append(lap_data['lap'])
                    y_data.append(lap_data['positions'][driver])
            lines[driver].set_data(x_data,y_data)
        lap_text = f"Lap: {min(frame+1,len(position_data))}/{len(position_data)}"
        ax.text(0.02,0.98,lap_text,transform=ax.transAxes,fontsize=14,color='white',weight='bold',bbox=dict(boxstyle='round',facecolor='red',alpha=0.8))
        return list(lines.values())
    
    anim = animation.FuncAnimation(fig,animate,frames=len(position_data),interval=200,blit=False,repeat=True)

    plt.tight_layout()
    return anim,fig


In [ ]:
# GENERATE ANIMATION

try:
    event_name = event['EventName']
    print(f"Creating animation for {event_name}...")
    anim,fig = create_race_animation(position_data,driver_info,event_name)
    plt.show()

    print("Animation Successful")
    print("\nTo Save to GIF, run next cell")
except Exception as e:
    print(f"Error creating animation {e}")

In [ ]:
# EXPORT GIF

def export_animation_gif(animation_obj,filename='f1_race_animation.gif',fps=4):
    try:
        print(f"Exporting animation as GIF: {filename}")
        animation_obj.save(filename,writer='pillow',fps=fps,savefig_kwargs={'facecolor': 'black'})

        return True
    except Exception as e:
        print(f"Error saving animation {e}")
        return False

try:
    if 'anim' in locals():
        success = export_animation_gif(anim,fps=4)
        if success:
            print("\nGIF Export Options:")
            print("Lower fps (1-3)")
            print("Higher fps (5-10)")
    else:
        print("No animation found")
except Exception as e:
    print(f"Error in export process {e}")

def export_custom_gif(animation_obj, filename, fps=4, dpi=80, optimize=True):
    """Export GIF with custom quality settings"""
    try:
        print(f"Exporting custom GIF: {filename}")

        # Custom writer with optimization
        writer = animation.PillowWriter(fps=fps)
        animation_obj.save(filename, writer=writer, dpi=dpi,
                          savefig_kwargs={'facecolor': 'black', 'bbox_inches': 'tight'})

        print(f"Custom GIF saved: {filename}")
        return True
    except Exception as e:
        print(f"Custom export failed: {e}")
        return False

print("\nCustom Export Options:")
print("Run this for different quality settings:")
print("export_custom_gif(anim, 'f1_high_quality.gif', fps=6, dpi=100)")
print("export_custom_gif(anim, 'f1_low_size.gif', fps=2, dpi=60)")

In [ ]:
# STATIC POSITION CHART

def create_static_position_chart(position_data,driver_info,event_name):
    fig, ax = plt.subplots(figsize=(16, 10))
    fig.patch.set_facecolor('black')

    all_drivers = list(driver_info.keys())
    colors = {}

    color_palette = plt.cm.Set3(np.linspace(0, 1, len(all_drivers)))

    for i, driver in enumerate(all_drivers):
        team_color = driver_info[driver].get('color', None)
        if team_color and team_color.startswith('#'):
            colors[driver] = team_color
        else:
            colors[driver] = color_palette[i]

    for driver in all_drivers:
        x_data = []
        y_data = []

        for lap_data in position_data:
            if driver in lap_data['positions']:
                x_data.append(lap_data['lap'])
                y_data.append(lap_data['positions'][driver])

        if x_data:  
            ax.plot(x_data, y_data, linewidth=2.5,
                   label=driver_info[driver]['abbreviation'],
                   color=colors[driver], alpha=0.8, marker='o', markersize=3)

    ax.set_xlim(1, len(position_data))
    ax.set_ylim(0.5, 20.5)
    ax.invert_yaxis()
    ax.set_xlabel('Lap Number', fontsize=12, color='white')
    ax.set_ylabel('Position', fontsize=12, color='white')
    ax.set_title(f'{event_name} - Complete Race Positions', fontsize=16, color='white', pad=20)
    ax.grid(True, alpha=0.3)
    ax.tick_params(colors='white')

    for spine in ax.spines.values():
        spine.set_color('white')

    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left',
             frameon=True, facecolor='black', edgecolor='white')

    plt.tight_layout()
    plt.show()

    return fig

try:
    static_fig = create_static_position_chart(position_data,driver_info,event['EventName'])
    print("Static Position Chart Created")
except Exception as e:
    print(f"Error creating static chart {e}")


In [ ]:
# RACE STATS

def print_race_statistics(position_data, driver_info, race_session):
    
    print("=== RACE STATISTICS ===\n")

    
    final_positions = position_data[-1]['positions']
    winner = min(final_positions.items(), key=lambda x: x[1])
    print(f"Race Winner: {driver_info[winner[0]]['full_name']} ({driver_info[winner[0]]['team']})")

    
    sorted_final = sorted(final_positions.items(), key=lambda x: x[1])
    print(f"Second Place: {driver_info[sorted_final[1][0]]['full_name']}")
    print(f"Third Place: {driver_info[sorted_final[2][0]]['full_name']}")

    print("\n=== POSITION CHANGES ===")

    
    position_changes = {}
    start_positions = position_data[0]['positions']

    for driver in final_positions:
        if driver in start_positions:
            change = start_positions[driver] - final_positions[driver]
            position_changes[driver] = change

    if position_changes:
        biggest_climber = max(position_changes.items(), key=lambda x: x[1])
        biggest_faller = min(position_changes.items(), key=lambda x: x[1])

        if biggest_climber[1] > 0:
            print(f"Biggest Climber: {driver_info[biggest_climber[0]]['full_name']} (+{biggest_climber[1]} positions)")

        if biggest_faller[1] < 0:
            print(f"Biggest Faller: {driver_info[biggest_faller[0]]['full_name']} ({biggest_faller[1]} positions)")

    print(f"\n=== RACE INFO ===")
    print(f"Total Laps: {len(position_data)}")
    print(f"Drivers: {len(driver_info)}")

    leaders = []
    for lap_data in position_data:
        leader = min(lap_data['positions'].items(), key=lambda x: x[1])
        leaders.append(leader[0])

    unique_leaders = list(set(leaders))
    print(f"Different Race Leaders: {len(unique_leaders)}")

    if len(unique_leaders) > 1:
        print("Race Leaders:")
        for leader in unique_leaders:
            laps_led = leaders.count(leader)
            print(f"  - {driver_info[leader]['full_name']}: {laps_led} laps")

try:
    print_race_statistics(position_data, driver_info, race)
except Exception as e:
    print(f"Error generating statistics: {e}")
